In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from os import listdir, path
import numpy as np
import random
from collections import defaultdict
import datetime

seq_length = 36
dim = 6


In [2]:
def padding(data):
    """Get neighboor padding."""
    padded_data = []
    noise_level = [ 20, 20, 20, 0.2, 0.2, 0.2 ]
    
    # Before- Neighbour padding
    tmp_data = (np.random.rand(seq_length, dim) - 0.5) * noise_level + data[0]
    tmp_data[(seq_length -
              min(len(data), seq_length)):] = data[:min(len(data), seq_length)]
    padded_data.append(tmp_data)
    # After- Neighbour padding
    tmp_data = (np.random.rand(seq_length, dim) - 0.5) * noise_level + data[-1]
    tmp_data[:min(len(data), seq_length)] = data[:min(len(data), seq_length)]
    padded_data.append(tmp_data)
    return padded_data

In [3]:
def build_dataset(data, label):
    """Support function for format.(Helps format train, valid and test.)"""
    # Add 2 padding, initialize data and label
    padded_num = 2
    length = len(data) * padded_num
    features = np.zeros((length, seq_length, dim))
    labels = np.zeros(length)
    # Get padding for train, valid and test
    for idx, (data, label) in enumerate(zip(data, label)):
        padded_data = padding(data)
        for num in range(padded_num):
            features[padded_num * idx + num] = padded_data[num]
            labels[padded_num * idx + num] = label
    # Turn into tf.data.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((features, labels.astype("int32")))
    return length, dataset

In [4]:
def time_warping(molecule, denominator, data):
  """Generate (molecule/denominator)x speed data."""
  tmp_data = [[0
               for i in range(len(data[0]))]
              for j in range((int(len(data) / molecule) - 1) * denominator)]
  for i in range(int(len(data) / molecule) - 1):
    for j in range(len(data[i])):
      for k in range(denominator):
        tmp_data[denominator * i +
                 k][j] = (data[molecule * i + k][j] * (denominator - k) +
                          data[molecule * i + k + 1][j] * k) / denominator
  return tmp_data


In [5]:
def augment_data(original_data, original_label):
  """Perform data augmentation."""
  new_data = []
  new_label = []
  for idx, (data, label) in enumerate(zip(original_data, original_label)):  # pylint: disable=unused-variable
    # Original data
    new_data.append(data)
    new_label.append(label)
    # Sequence shift
    for num in range(5):  # pylint: disable=unused-variable
      new_data.append((np.array(data, dtype=np.float32) +
                       (random.random() - 0.5) * 200).tolist())
      new_label.append(label)
    # Random noise
    tmp_data = [[0 for i in range(len(data[0]))] for j in range(len(data))]
    for num in range(5):
      for i in range(len(tmp_data)):
        for j in range(len(tmp_data[i])):
          tmp_data[i][j] = data[i][j] + 5 * random.random()
      new_data.append(tmp_data)
      new_label.append(label)
    # Time warping
    fractions = [(3, 2), (5, 3), (2, 3), (3, 4), (9, 5), (6, 5), (4, 5)]
    for molecule, denominator in fractions:
      new_data.append(time_warping(molecule, denominator, data))
      new_label.append(label)
    # Movement amplification
    for molecule, denominator in fractions:
      new_data.append(
          (np.array(data, dtype=np.float32) * molecule / denominator).tolist())
      new_label.append(label)
  return new_data, new_label

In [6]:
def load_data(data_type, files):
    data   = []
    labels = []
    random.shuffle(files)
    
    for file in files:
        with open(file) as f:
            label = path.splitext(file)[0][-1]
            labels.append(label)
            readings = []
            for line in f:
                reading = line.strip().split(',')
                readings.append([float(i) for i in reading[0:6]])

            data.append(readings)
            
    if data_type == 'train':
        data, labels = augment_data(data, labels)
    
    return build_dataset(data, labels)

In [7]:
import math

files_path = defaultdict(list)
dir = './data'
for filename in listdir(dir):
    if filename.endswith('.csv'):
        digit = path.splitext(filename)[0][-1]
        files_path[digit].append(path.join(dir, filename))

train_files      = []
validation_files = []
test_files       = []

for digit in files_path:
    random.shuffle(files_path[digit])
    
    train_split = math.floor(len(files_path[digit]) * 0.6) # 60%
    validation_split = train_split + math.floor(len(files_path[digit]) * 0.2) # 20%

    train_files += files_path[digit][:train_split]
    validation_files += files_path[digit][train_split:validation_split]
    # remaining 20%
    test_files += files_path[digit][validation_split:]

train_length, train_data = load_data('train', train_files)
validation_length, validation_data = load_data('validation', validation_files)
test_length, test_data = load_data('test', test_files )

#print(train_length)

for (ds, lb) in train_data.take(1):
    print(ds)

tf.Tensor(
[[ 8.98990727e+02  2.51573364e+02  3.10571131e+02  2.70816742e+01
   5.85281014e+00 -1.74726738e+01]
 [ 9.08521538e+02  2.39759341e+02  3.17019650e+02  2.69394048e+01
   5.79981650e+00 -1.75068782e+01]
 [ 8.96105965e+02  2.39900242e+02  3.14267812e+02  2.70668969e+01
   5.78631900e+00 -1.74236732e+01]
 [ 9.03117475e+02  2.36264405e+02  3.21922351e+02  2.69413800e+01
   5.85296206e+00 -1.75274578e+01]
 [ 9.04556803e+02  2.49923846e+02  3.25924854e+02  2.69415929e+01
   5.75412488e+00 -1.75472434e+01]
 [ 9.03320000e+02  2.45610000e+02  3.19340000e+02  2.69900000e+01
   5.82000000e+00 -1.74800000e+01]
 [ 9.61910000e+02  1.83110000e+02  2.19730000e+02  2.51100000e+01
   1.35700000e+01 -2.56000000e+01]
 [ 1.17383000e+03 -1.80180000e+02  8.33980000e+02 -1.62000000e+01
  -4.31300000e+01  4.16900000e+01]
 [ 9.31150000e+02 -7.34380000e+02  2.81740000e+02 -1.50000000e+00
  -2.78900000e+01  5.55300000e+01]
 [ 4.41410000e+02 -7.63180000e+02 -1.20996000e+03  7.78000000e+00
  -3.06900000e

In [8]:
model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(8, (4, 3), padding="same", activation="relu", input_shape=(seq_length, dim, 1)),
      tf.keras.layers.MaxPool2D((3, 3)),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Conv2D(16, (4, 1), padding="same", activation="relu"),
      tf.keras.layers.MaxPool2D((3, 1), padding="same"),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(16, activation="relu"),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(10, activation="softmax")
  ])

In [9]:
epochs = 50
batch_size = 64

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

def reshape_function(data, label):
  reshaped_data = tf.reshape(data, [-1, 6, 1])
  return reshaped_data, label

train_data = train_data.map(reshape_function)
validation_data = validation_data.map(reshape_function)

train_data = train_data.batch(batch_size).repeat()
validation_data = validation_data.batch(batch_size)

logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
  train_data,
  epochs=epochs,
  validation_data=validation_data,
  steps_per_epoch=1000,
  validation_steps=int((validation_length - 1) / batch_size + 1),
  callbacks=[tensorboard_callback])

Train for 1000 steps, validate for 1 steps
Epoch 1/50
1000/1000 [==============================] - 4s 4ms/step - loss: 10.5381 - accuracy: 0.0641 - val_loss: 2.3030 - val_accuracy: 0.1000
Epoch 2/50
1000/1000 [==============================] - 4s 4ms/step - loss: 2.3062 - accuracy: 0.0309 - val_loss: 2.3028 - val_accuracy: 0.1000
Epoch 3/50
1000/1000 [==============================] - 4s 4ms/step - loss: 2.3009 - accuracy: 0.0240 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 4/50
1000/1000 [==============================] - 4s 4ms/step - loss: 2.2966 - accuracy: 0.0860 - val_loss: 2.3027 - val_accuracy: 0.1000
Epoch 5/50
1000/1000 [==============================] - 4s 4ms/step - loss: 2.2911 - accuracy: 0.1081 - val_loss: 2.3028 - val_accuracy: 0.1000
Epoch 6/50
1000/1000 [==============================] - 4s 4ms/step - loss: 2.2745 - accuracy: 0.1150 - val_loss: 2.2306 - val_accuracy: 0.1250
Epoch 7/50
1000/1000 [==============================] - 4s 4ms/step - loss: 2.2531 - accurac

In [10]:
test_data = test_data.map(reshape_function)
test_labels = np.zeros(test_length)

idx = 0
for data, label in test_data:
    test_labels[idx] = label.numpy()
    idx += 1
    
test_data = test_data.batch(batch_size)

loss, acc = model.evaluate(test_data)
pred = np.argmax(model.predict(test_data), axis=1)
confusion = tf.math.confusion_matrix(labels=tf.constant(test_labels), predictions=tf.constant(pred), num_classes=10)
print(confusion)


1/1 [==============================] - 0s 3ms/step - loss: 1.6854 - accuracy: 0.5000
tf.Tensor(
[[2 0 0 0 2 0 0 0 0 0]
 [0 0 1 0 1 1 1 0 0 0]
 [0 0 4 0 0 0 0 0 0 0]
 [0 0 1 3 0 0 0 0 0 0]
 [0 0 1 0 1 2 0 0 0 0]
 [0 0 0 0 0 1 2 0 0 1]
 [0 0 0 0 0 0 4 0 0 0]
 [0 1 0 1 0 0 0 2 0 0]
 [0 0 0 0 1 1 0 0 2 0]
 [0 2 0 0 0 0 1 0 0 1]], shape=(10, 10), dtype=int32)
